<a href="https://colab.research.google.com/github/jclutian/inf2178-expdesignfordatascience-w23/blob/main/mid-term/Group68/group68-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>